# Import

/!\ Tensorflow < 2.0 is needed
!conda install tensorflow=1.11.0

In [3]:
import os
import glob
import numpy as np
import pprint

import tensorflow as tf

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Utils and UI functions

In [4]:
global request_results

all_logs = {}
all_logs_stats = {}
request_results = []

tag_list = []
target_list = ["max", "min", "mean", "std", "last"]

In [11]:
def get_all_tag(path):
    tag_list = []

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag not in tag_list:
                    tag_list.append(value.tag)
    
    return tag_list

In [12]:
def get_log_data(tags, path):
    logs = dict(zip(tags, [[] for _ in range(len(tags))]))

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag not in tag_list:
                    tag_list.append(value.tag)
                logs[value.tag].append(value.simple_value)
    
    return logs

In [13]:
# search in the stat
def get_of(all_logs_stats, tag = tag_list):
    global request_results
    
    def get_of_ui(key = target_list, search = ""):
        global request_results
        request_results = []

        for path in all_logs_stats.keys():
            if search != "":
                if search in path:
                    request_results.append(all_logs_stats[path][tag][key])
            else:
                request_results.append((path, all_logs_stats[path][tag][key]))
        print("done")
            
    interact_manual(get_of_ui)

In [14]:
def fetch_logs(directory, tag):

    all_logs = {}
    all_logs_stats = {}
    request_results = []

    # ---- get the path for all logs file ----
    all_log_dir = os.listdir(directory)

    all_log_path = []
    for log_dir in all_log_dir:
        log_dir_path = os.path.join(directory, log_dir)
        
        if os.path.isdir(log_dir_path):
            all_log_path.append(os.path.join(log_dir_path, os.listdir(log_dir_path)[0]))

    # ---- recover all the logs ----
    all_logs = dict(zip(all_log_path, [None for _ in range(len(all_log_path))]))

    # for path in all_log_path:
    for path in all_log_path:
        all_logs[path] = get_log_data([tag], path)


    # ---- compute the statistics ----
    all_logs_stats = dict(zip(all_log_path, [{} for _ in range(len(all_log_path))]))

    for path in all_log_path:
        for tag in all_logs[path]:
            try: 
                all_logs_stats[path][tag] = {"max": None, "min": None, "mean": None, "std": None, "last": None}
                all_logs_stats[path][tag]["max"] = np.asarray(all_logs[path][tag]).max()
                all_logs_stats[path][tag]["min"] = np.asarray(all_logs[path][tag]).min()
                all_logs_stats[path][tag]["mean"] = np.asarray(all_logs[path][tag]).mean()
                all_logs_stats[path][tag]["std"] = np.asarray(all_logs[path][tag]).std()
                all_logs_stats[path][tag]["last"] = np.asarray(all_logs[path][tag])[-1]
                
            except ValueError as e:
                print("Error for tag %s in path %s" % (tag, path))
                print("continue...")
                pass

    return all_logs_stats

In [16]:
tensorboard_root = "../tensorboard/"

def first_level_logs():
    list_dir = glob.glob("../tensorboard/*/*")
    list_dir = sorted(list_dir)
    return list_dir

def log_fetcher():
    first_level_directory = None
    second_level_directory = None
    selected_tag = None
    
def first_level_directory(directory):
    list_dir = glob.glob(directory + "/*")
    list_dir = sorted(list_dir)
    interact(second_level_directory, directory=list_dir)
    
def second_level_directory(directory):
    all_log_stat = fetch_logs(directory, [])
    interact(third_level_tag, tag=tag_list)
    
def third_level_tag(tag=tag_list):
    all_log_stat = fetch_logs(directory, tag)
    get_of(all_log_stat, tag)
    
interact(first_level_directory, directory=first_level_logs())
        

interactive(children=(Dropdown(description='directory', options=('../tensorboard/osirim_0.5r_cnn_crossval_dct/…

<function __main__.first_level_directory(directory)>

# UI

In [15]:
response = request_results
# trim path from everythinfg irelevant
# response = [(response[i][0].split("/")[4].split("_")[4][:4], response[i][1]) for i in range(len(response))]

# remove none
response = [(r[0], r[1]) for r in response if r[1] is not None]
# order by run
response = sorted(response, key=lambda x: x[1])

values = np.array([v[1] for v in response])
path = [p[0] for p in response]

values = np.array([i for i in values if i is not None])
print("Overall statistics --------")
print("mean: ", np.nanmean(values))
print("std:  ", np.nanstd(values))
# print("min:  ", values.min(), 2)
# print("max:  ", values.max(), 2)
print("")
print("%s values --------")
for p, v in response:
    print(p, v)

Overall statistics --------
mean:  nan
std:   nan

%s values --------


/home/lcances/.miniconda3/envs/dcase2020/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
